In [2]:
import numpy as np
from scipy import stats as st
from matplotlib import pyplot as plt

%matplotlib inline

In [3]:
n = 100
teta = 2

Генерация выборки

In [4]:
data = np.power(1 - np.random.rand(n), 1 / (1 - teta))
data

array([  3.89917144,   5.80051728,   2.43028806,   1.0499324 ,
         1.50362749,   3.06013165,   1.68350057,   3.74060098,
         6.22714988,   2.7445833 ,   1.17098713,   1.4801626 ,
         1.83938168,   1.61917698,   1.12889764,   1.08761507,
         1.40741645,   1.72899448,   1.52958938,   1.12809531,
         8.16130567,   3.17948918,   2.78451182,   1.43805052,
         1.7683587 ,   1.86408907,   1.48392306,   1.5872687 ,
         1.24221452,   4.23412759,   1.99102612,   3.03881212,
         1.49234591,   1.22691133,   2.93133562,   6.47417285,
         1.5247843 ,   3.85359147,   1.8160591 ,   3.23764175,
        72.46039633,   1.41654381,   1.27931299,   1.34405141,
         2.85115812,   2.07417971,   1.64521943,   1.09361539,
         1.62383782,   1.00203313,   3.40534425,   1.59440389,
        12.52890808,   2.49007431,   1.45207119,   1.2631932 ,
         3.8889904 ,   1.36086681,   2.13874409,   2.68739886,
         9.14669331,  47.19729496,   1.27130745,   4.79

вычисление полученных аналитически доверительных интервалов

In [5]:
beta = 0.95
t1 = -1.96
t2 = 1.96

оценка

In [6]:
assessment_teta = (n / np.sum(np.log(data))) + 1
assessment_teta

2.0172272844604375

In [7]:
h1 = assessment_teta + t1 * (assessment_teta - 1) / n**0.5
h2 = assessment_teta + t2 * (assessment_teta - 1) / n**0.5
print(f"h1 - {h1}\nh2 - {h2}\nдлина интервала - {h2 - h1}")
an_res_teta = (h1,h2)

h1 - 1.8178507367061918
h2 - 2.2166038322146835
длина интервала - 0.3987530955084917


оценка медианы

In [8]:
assessment_med = np.power(2, 1 / (assessment_teta - 1))
assessment_med

1.9766596307022708

In [9]:
h1 = t1 * assessment_med * np.log(2) / (assessment_teta - 1) / np.sqrt(n) + assessment_med
h2 = t2 * assessment_med * np.log(2) / (assessment_teta - 1) / np.sqrt(n) + assessment_med
print(f"h1 - {h1}\nh2 - {h2}\nдлина интервала - {h2 - h1}")
an_res_med = (h1,h2)

h1 - 1.71266479897473
h2 - 2.2406544624298115
длина интервала - 0.5279896634550816


Непараметрический бутстрап для оценки теты

In [10]:
func = lambda x: (n / np.sum(np.log(x))) + 1 - assessment_teta
non_parametric_bootstrap_teta = st.bootstrap(
    (data, ), 
    func, 
    n_resamples=1_000, 
    method='basic',
    confidence_level=0.95
).confidence_interval

non_parametric_bootstrap_teta = (assessment_teta - non_parametric_bootstrap_teta.high, assessment_teta - non_parametric_bootstrap_teta.low)
print("доверительный интервал - ",non_parametric_bootstrap_teta, "\nдлина интервала -",non_parametric_bootstrap_teta[1] - non_parametric_bootstrap_teta[0])

доверительный интервал -  (1.8661863260403126, 2.2410012524014027) 
длина интервала - 0.37481492636109004


Параметрический бутстрап для оценки теты

In [11]:
from collections.abc import Callable
from typing import NamedTuple


class ConfidenceInterval(NamedTuple):
    low: float
    high: float


def parametric_bootstrap(
    data: np.ndarray,
    statistic: Callable[[np.ndarray], float],
    prob_model: Callable[[int], np.ndarray],
    n_resamples: int = 10_000,
    confidence_level: float = 0.95,
) -> ConfidenceInterval:
    n = len(data)
    bootstrap_data = np.ndarray((n_resamples, ), dtype=np.float32)

    for i in range(n_resamples):
        resample = prob_model(n)
        bootstrap_data[i] = statistic(resample)
    
    bootstrap_data.sort()

    return ConfidenceInterval(
        low=bootstrap_data[round((1 - confidence_level) / 2 * n_resamples) - 1],
        high=bootstrap_data[round((1 + confidence_level) / 2 * n_resamples) - 1]
    )


parametric_bootstrap_teta = parametric_bootstrap(
    data,
    func,
    lambda n: np.power((1 - np.random.rand(n)), 1 / (1 - assessment_teta)),
    n_resamples=50_000, 
    confidence_level=0.95
)

parametric_bootstrap_teta = (assessment_teta - parametric_bootstrap_teta.high, assessment_teta - parametric_bootstrap_teta.low)
print("доверительный интервал - ",parametric_bootstrap_teta, "\nдлина интервала -",parametric_bootstrap_teta[1] - parametric_bootstrap_teta[0])

доверительный интервал -  (1.7847741915277364, 2.1906579656890752) 
длина интервала - 0.4058837741613388


Непараметрический бутстрап для оценки медианы

In [12]:
func = lambda x: np.power(2, 1 / (n / np.sum(np.log(x)))) - assessment_med
non_parametric_bootstrap_med = st.bootstrap(
    (data, ), 
    func, 
    n_resamples=1_000, 
    method='basic',
    confidence_level=0.95
).confidence_interval

non_parametric_bootstrap_med  = (assessment_med - non_parametric_bootstrap_med .high, assessment_med - non_parametric_bootstrap_med .low)
print("доверительный интервал - ",non_parametric_bootstrap_med, "\nдлина интервала -",non_parametric_bootstrap_med[1] - non_parametric_bootstrap_med[0])

доверительный интервал -  (1.7522965436147142, 2.2456136084953253) 
длина интервала - 0.4933170648806111


Параметрический бутстрап для оценки медианы

In [13]:
from collections.abc import Callable
from typing import NamedTuple


class ConfidenceInterval(NamedTuple):
    low: float
    high: float


def parametric_bootstrap(
    data: np.ndarray,
    statistic: Callable[[np.ndarray], float],
    prob_model: Callable[[int], np.ndarray],
    n_resamples: int = 10_000,
    confidence_level: float = 0.95,
) -> ConfidenceInterval:
    n = len(data)
    bootstrap_data = np.ndarray((n_resamples, ), dtype=np.float32)

    for i in range(n_resamples):
        resample = prob_model(n)
        bootstrap_data[i] = statistic(resample)
    
    bootstrap_data.sort()

    return ConfidenceInterval(
        low=bootstrap_data[round((1 - confidence_level) / 2 * n_resamples) - 1],
        high=bootstrap_data[round((1 + confidence_level) / 2 * n_resamples) - 1]
    )


parametric_bootstrap_med = parametric_bootstrap(
    data,
    func,
    lambda n: np.power(np.random.rand(n), 1 / (1 - assessment_teta)),
    n_resamples=50_000, 
    confidence_level=0.95
)

parametric_bootstrap_med = (assessment_med - parametric_bootstrap_med.high, assessment_med - parametric_bootstrap_med.low)
print("доверительный интервал - ",parametric_bootstrap_med, "\nдлина интервала -",parametric_bootstrap_med[1] - parametric_bootstrap_med[0])

доверительный интервал -  (1.6795738090737002, 2.208889042304291) 
длина интервала - 0.5293152332305906


Сравнение

In [14]:
print(
    "Для теты",
    f"{an_res_teta = }",
    f"{non_parametric_bootstrap_teta = }",
    f"{parametric_bootstrap_teta = }",
    "\nДля медианы",
    f"{an_res_med = }",
    f"{non_parametric_bootstrap_med = }",
    f"{parametric_bootstrap_med = }",
    sep="\n\n"
)

Для теты

an_res_teta = (1.8178507367061918, 2.2166038322146835)

non_parametric_bootstrap_teta = (1.8661863260403126, 2.2410012524014027)

parametric_bootstrap_teta = (1.7847741915277364, 2.1906579656890752)


Для медианы

an_res_med = (1.71266479897473, 2.2406544624298115)

non_parametric_bootstrap_med = (1.7522965436147142, 2.2456136084953253)

parametric_bootstrap_med = (1.6795738090737002, 2.208889042304291)
